In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
# Import the time module so we can time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").config("spark.driver.memory", "2g").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/15 14:14:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/3/NYC_Building_Violations.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("NYC_Building_Violations.csv"), sep=",", header=True)
df.show()

+----------------+----+-------+-----+-----+----------+-------------------+----------------+------------+--------------------+----------------+--------------------+-------------+--------------------+----------+--------------------+--------------------+--------------------+
|ISN_DOB_BIS_VIOL|BORO|    BIN|BLOCK|  LOT|ISSUE_DATE|VIOLATION_TYPE_CODE|VIOLATION_NUMBER|HOUSE_NUMBER|              STREET|DISPOSITION_DATE|DISPOSITION_COMMENTS|DEVICE_NUMBER|         DESCRIPTION|ECB_NUMBER|              NUMBER|  VIOLATION_CATEGORY|      VIOLATION_TYPE|
+----------------+----+-------+-----+-----+----------+-------------------+----------------+------------+--------------------+----------------+--------------------+-------------+--------------------+----------+--------------------+--------------------+--------------------+
|         2286033|   1|1009713|00577|00019|  20180507|                  E|     9027/627971|          34|        WEST 14TH ST|        20220509|PPN203 AOC SUB 05...|      1P13420|    

In [4]:
# Get a summary of the data. 


+-------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+----------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+
|summary|  ISN_DOB_BIS_VIOL|              BORO|               BIN|             BLOCK|               LOT|         ISSUE_DATE|VIOLATION_TYPE_CODE|VIOLATION_NUMBER|      HOUSE_NUMBER|            STREET|    DISPOSITION_DATE|DISPOSITION_COMMENTS|       DEVICE_NUMBER|         DESCRIPTION|          ECB_NUMBER|            NUMBER|  VIOLATION_CATEGORY|      VIOLATION_TYPE|
+-------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+----------------+------------------+------------------+--------------------+--------------------+--------------------+------

In [5]:
 # Let's create a view with our DataFrame and run SQL that will sum up the boroughs by the type of violation.
# We can output the time this step runs in seconds.
# Because we are timing the executions, remember to run twice to eliminate the "load time" from the discussion.






+--------------------+---------+
|      VIOLATION_TYPE|sum(BORO)|
+--------------------+---------+
|LL10/80-LOCAL LAW...|   3609.0|
|LL11/98-LOCAL LAW...|   9285.0|
|HVIOS-NYCHA ELEV ...|    969.0|
|P-PLUMBING       ...|  29480.0|
|ACH1-(NYCHA) - EL...|   4949.0|
|LANDMRK-LANDMARK ...|   5599.0|
|LL5-LOCAL LAW 5/7...|   1363.0|
|IMD-IMMEDIATE EME...|     13.0|
|B-BOILER         ...|  17042.0|
|FISP-FACADE SAFET...|   6889.0|
|EGNCY-EMERGENCY  ...|  12607.0|
|ES-ELECTRIC SIGNS...|  18378.0|
|                null|    148.0|
|L1198-LOCAL LAW 1...|  10656.0|
|HBLVIO-HIGH PRESS...|  14628.0|
|BENCH-FAILURE TO ...| 110285.0|
|RWNRF-RETAINING W...|   4007.0|
|FISPNRF-NO REPORT...|  21017.0|
|LL2604-PHOTOLUMIN...|    679.0|
|LL2604S-SPRINKLER...|   1513.0|
+--------------------+---------+
only showing top 20 rows

--- 2.8327789306640625 seconds ---


In [6]:
# Write out the data in parquet format
# Note: That this is pretty much the same as writing out to a csv to your local directory.
# We are telling Spark to overwrite all of the data if it already exists




*   click the folder icon on the left of the notebook to expose the folders and files stored in your colab enviornment.  Notice that a new folder is present with the same name as your parquet file (parquet_title_basic)
*   inside of it you will find 'part-*.parquet' files and a '_SUCCESS' file. 
*  The '_SUCCESS' file is created when Spark creates a Parquet folder
*  the part-* files are binary files that store your compressed data in columnar format





In [7]:
# Read in our new parquet formatted data


In [8]:
# A parquet formatted DataFrame has all the same methods as a row-based DataFrame
# We can convert the DataFrame to a view.


In [9]:
# Run the same sql as above.  (Note: If you have small datasets it IS possible that times may be very close.)
# Because we are timing the executions, remember to run twice to eliminate the "load time" from the discussion.



+--------------------+---------+
|      VIOLATION_TYPE|sum(BORO)|
+--------------------+---------+
|LL10/80-LOCAL LAW...|   3609.0|
|LL11/98-LOCAL LAW...|   9285.0|
|HVIOS-NYCHA ELEV ...|    969.0|
|P-PLUMBING       ...|  29480.0|
|ACH1-(NYCHA) - EL...|   4949.0|
|LANDMRK-LANDMARK ...|   5599.0|
|LL5-LOCAL LAW 5/7...|   1363.0|
|IMD-IMMEDIATE EME...|     13.0|
|B-BOILER         ...|  17042.0|
|FISP-FACADE SAFET...|   6889.0|
|EGNCY-EMERGENCY  ...|  12607.0|
|ES-ELECTRIC SIGNS...|  18378.0|
|                null|    148.0|
|L1198-LOCAL LAW 1...|  10656.0|
|HBLVIO-HIGH PRESS...|  14628.0|
|BENCH-FAILURE TO ...| 110285.0|
|RWNRF-RETAINING W...|   4007.0|
|FISPNRF-NO REPORT...|  21017.0|
|LL2604-PHOTOLUMIN...|    679.0|
|LL2604S-SPRINKLER...|   1513.0|
+--------------------+---------+
only showing top 20 rows

--- 1.0396668910980225 seconds ---


In [11]:
# Writing out a csv file from Spark will also create a folder with "part" files.
# These files are not binary or compressed and in reality are just normal csv files broken into partitions.
# You can see the folder 'out_violations.csv' in your local directory.


### Read a parquet file into a Pandas DataFrame

In [12]:
import pandas as pd

In [14]:
# Open the parquet_violations folder and get the name of a file and edit the path to the parquet file.  
# Check for the correct file name, since the file number will change.


# Convert the parquet file to a Pandas DataFrame. 


,ISN_DOB_BIS_VIOL,BORO,BIN,BLOCK,LOT,ISSUE_DATE,VIOLATION_TYPE_CODE,VIOLATION_NUMBER,HOUSE_NUMBER,STREET,DISPOSITION_DATE,DISPOSITION_COMMENTS,DEVICE_NUMBER,DESCRIPTION,ECB_NUMBER,NUMBER,VIOLATION_CATEGORY,VIOLATION_TYPE
0,2286033,1,1009713,00577,00019,20180507,E,9027/627971,34,WEST 14TH ST,20220509,PPN203 AOC SUB 050322 BY BP ELEV CO ...,1P13420,None,None,V*050718E9027/627971,V*-DOB VIOLATION - Resolved,E-ELEVATOR ...
1,2533639,1,1082666,00333,00001,20210629,E,9027/705433,77,COLUMBIA STREET,20220509,PPN203 AOC SUB 050222 BY MIDTOWN ELEV CO INC ...,1P27474,None,None,V*062921E9027/705433,V*-DOB VIOLATION - Resolved,E-ELEVATOR ...
2,2347979,1,1083846,01130,00001,20190423,E,9028/648125,200,CENTRAL PARK WEST,20220509,PPN203 AOC SUBMITTED ON 050522 BY CENTENNIAL E...,1P40861,None,None,V*042319E9028/648125,V*-DOB VIOLATION - Resolved,E-ELEVATOR ...
3,2566336,1,1057155,01889,07502,20211123,E,9028/710097,845,WEST END AVE,20220509,PPN203 AOC SUB 050322 BY BP ELEV CO ...,1P14972,None,None,V*112321E9028/710097,V*-DOB VIOLATION - Resolved,E-ELEVATOR ...
4,2487351,1,1041456,01387,00021,20200925,E,9028/689200,31,E 72 ST,20220509,PPN203 AOC SUB 050322 BY BP ELEV CO ...,1P10910,None,None,V*092520E9028/689200,V*-DOB VIOLATION - Resolved,E-ELEVATOR ...
